### 将标记数据文本按照。拆分成多条标记数据

In [1]:
import re
import copy
result_labeled_data, text_field = [], 2
with open(r'data\data_xbs_mdlfeature.txt') as f:
    for line in f.readlines()[1:]:
        line_arr = line.strip().split('	')
        text_arr = line_arr[text_field].split('。')
        for s in text_arr:
            if s != '':
                t_arr = copy.deepcopy(line_arr)
                t_arr[text_field] = s + '。'
                result_labeled_data.append(t_arr)
print('%s lines loaded!' % len(result_labeled_data))

11921 lines loaded!


### 使用正则来将文本拆分为短句，并赋予类别标签

In [2]:
import random

def classify_text_snippet(labeled_data, label_field, regex, print_flag='2'):
    """
    使用正则来拆分文本句子，将包含正则的短句划分到相应的标记类别，没有正则的短句划分到相应的标记类别
    """
    sents = {'0': [], '1': [], '2': [], '3': []}
    for line in labeled_data:
        text, flag = line[text_field], line[label_field]
        
        # 正则匹配上
        start, end = 1000, -1
        it = re.finditer(regex, text)
        for match in it:
            if match.span(0)[0] < start:
                start = match.span(0)[0]
            if match.span(0)[1] > end:
                end = match.span(0)[1]
        span = text[start:end] if end != -1 else ''
        if span != '':
            span = span.replace('(', '\(').replace(')', '\)').replace('^', '\^').replace('+', '\+').replace('?', '\?').replace('.', '\.')
            span = span.replace('*', '\*')
            match = re.search('[^，。,；]*' + span + '[^，。,；]*', text)
#             if match is None:
#                 print(text)
#                 print(span)
            s = match.group(0)
            sents[flag].append(s)
            text = text.replace(s, '')
            if flag == print_flag:
                print(line[0], s)
        
        arr = re.split('[，,；;]', text)
        for r in arr:
            if r != '':
                sents['3'].append(r + '，')

    for key in sents.keys():
        sents[key] = list(set(sents[key]))
        random.shuffle(sents[key])
        print(key, len(sents[key]))
        
    return sents

def generate_snippet_to_text(sents_dict):
    """
    将短句合并生成长文本
    """
    result = []
    for i in range(1000):
        for key in ['0', '1', '2']:
            s_arr = random.sample(sents_dict['3'], random.randint(3, 10))
            if key == '2':
                if random.randint(0, 10) == 0:
                    s = random.choice(sents_dict[key])
                else:
                    s = ''
            else:
                s = random.choice(sents_dict[key])
            s_arr.append(s)
            random.shuffle(s_arr)
            result.append(('，'.join(s_arr), key))

    print(len(result))
    return result

### 使用正则来筛选整句

In [3]:
def classif_text(labeled_data, label_field, regex):
    sents = {'0': [], '1': [], '2': [], '3': []}
    for line in labeled_data:
        text, flag = line[text_field], line[label_field]
        start, end = 1000, -1
        match = re.search(regex, text)
        if match:
            sents[flag].append(text)
        else:
            sents['3'].append(text)

    for key in sents.keys():
        sents[key] = list(set(sents[key]))
        print(key, len(sents[key]))
        
    return sents

def generate_text(sents_dict):
    result = []
    for i in range(1000):
        for key in ['0', '1', '2']:
            if key == '2':
                if random.randint(0, 3) == 0:
                    s = random.choice(sents_dict[key])
                else:
                    s = random.choice(sents_dict['3'])
            else:
                s = random.choice(sents_dict[key])
            s2 = random.choice(sents_dict['3'])
            arr = [s, s2]
            result.append((''.join(arr), key))

    print(len(result))
    return result

In [4]:
from run import train1, predict


def write_data_to_file(data, file_name, field_name):
    with open(r'data\%s.txt' % file_name, 'w', encoding='utf-8') as f:
        f.write('文本,%s\n' % field_name)
        for text, flag in data:
            f.write('%s,%s\n' % (text, flag))

### 放射痛（右肩、肩胛和背部）

In [5]:
sents1 = classify_text_snippet(result_labeled_data, 3, '放射', '2')
data1 = generate_snippet_to_text(sents1)
sents2 = classif_text(result_labeled_data, 3, '放射|肩|背')
data2 = generate_text(sents2)
write_data_to_file(data1 + data2, '放射痛_右肩', '放射痛_右肩')

10002192 放射至后背
10005033 有腰背部放射痛
10010721 向背部放射
10014098 伴后背放射痛
10015132 伴后背放射痛
10017235 伴腰背部放射
10024192 无腰背部放射痛
10030179 不向腰背部放射
10032606 伴腰背部放射
10033502 无腰背部放射痛
10034091 伴后背放射痛
10037144 疼痛向腰背部放射
10037989 无腰背部放射痛
10054640 向腰背部放射
10055680 疼痛向后背放射
10055694 向腰背部放射
10068883 逐渐放射至右侧腰背部
10073519 无左下腹放射痛
10088057 向腰背部放射
10093985 不伴胸痛、腰背放射痛
10094425 不向其他部位放射
10098446 放射腹部平片DR：腹部部分肠腔积气
10105038 伴右背部放射性疼痛
10112211 未向肩背部放射
10121253 疼痛向背部放射
10122455 我院门诊2021-02-28放射腹部平片DR：考虑不完全肠梗阻
10126833 无腰背部放射痛
10139381 无腰背部及会阴部放射痛
10142410 无腰背部放射痛
10142415 无腰背部放射痛
10147487 无腰背部放射痛
10149504 伴有背部放射痛
10149753 伴有腰背部放射痛
10150713 无腰背部放射痛
10151531 伴后背放射痛
10151729 无腰背部放射痛
10159623 偶有后背放射痛
10161009 伴腰背部及肩胛部放射痛
10173503 放射胸部正侧位DR未见异常
10174498 伴胸背部放射痛
10174964 向腰背部放射
10182612 向腰背部放射
10185220 不向腰背部放射
10188273 放射至背部
10204932 完善头胸腹部CT提示：1.两侧基底节及放射冠区腔隙性脑梗塞
20007984 不伴腰背部及胸部放射痛
20011315 疼痛不向其他部位放射
20016867 不伴腰背部放射痛
20022489 患者自述3天前在无明显诱因下出现间断性上腹部胀痛伴有后背部放射性疼痛
20028634 无腰背部放射性疼痛
20031081 患者自述1天前在无明显诱因下出现上腹部胀痛伴有后背部放射性疼痛
20043206

In [7]:
train1(file_path=r'data\放射痛_右肩.txt', text_field=0, label_field=1, log_file=r'output/logs/放射痛_右肩_train_20220812.txt')

DEBUG:root:Loading data lines...
DEBUG:root:6001 Lines Loaded!
DEBUG:root:Statistic Class Sample Number...
DEBUG:root:Class Sample Number: {'0': 2000, '1': 2000, '2': 2000}
INFO:root:Initializing Class TextClassifier...


0, total num: 2000, train num: 1600
1, total num: 2000, train num: 1600
2, total num: 2000, train num: 1600


Some weights of the model checkpoint at ../../BertModels/medical-roberta-wwm were not used when initializing BertModel: ['cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at ../../BertModels/medical-roberta-wwm and are newly initialized: ['bert.pooler.dense.weig

KeyboardInterrupt: 

In [6]:
best_mdl_path = r'output/models/放射痛_右肩_数据增强_20220816084317_9058.pth'
col, name = 3, '放射痛（右肩、肩胛和背部）'
predict(best_mdl_path, predict_file_path=r'data\测试集_现病史_模型.txt', result_file_path=r'output/data/%s_现病史_测试集结果.txt' % name, 
        log_file_path=r'output/data/predict_20220815_2.txt', num_fields=24, text_field=2, label_field=col, feature_name=name, skip_title=True)
predict(best_mdl_path, predict_file_path=r'data\人机_现病史_模型.txt', result_file_path=r'output/data/%s_现病史_人机结果.txt' % name, 
        log_file_path=r'output/data/predict_20220815_2.txt', num_fields=24, text_field=2, label_field=col, feature_name=name, skip_title=True)

DEBUG:root:Loading data lines...
DEBUG:root:689 Lines Loaded!
INFO:root:Initializing Class TextClassifier...
DEBUG:root:Loading Model output/models/放射痛_右肩_数据增强_20220816084317_9058.pth
INFO:root:TextClassifier Using Device: CUDA
INFO:root:TextClassifier Loading Data...
DEBUG:root:Using Text Pair? -> Yes
DEBUG:root:Data For Predicting
DEBUG:root:Loading data lines...
DEBUG:root:184 Lines Loaded!
INFO:root:Initializing Class TextClassifier...


best accuracy: 0.8318965517241379
{0: '0', 1: '1', 2: '2'}


DEBUG:root:Loading Model output/models/放射痛_右肩_数据增强_20220816084317_9058.pth
INFO:root:TextClassifier Using Device: CUDA
INFO:root:TextClassifier Loading Data...
DEBUG:root:Using Text Pair? -> Yes
DEBUG:root:Data For Predicting


best accuracy: 0.967391304347826
{0: '0', 1: '1', 2: '2'}


### 厌食

In [6]:
sents1 = classify_text_snippet(result_labeled_data, 5, '食', '2')
data1 = generate_snippet_to_text(sents1)
sents2 = classif_text(result_labeled_data, 5, '食')
data2 = generate_text(sents2)
write_data_to_file(data1 + data2, '厌食', '厌食')

10172369 3天前食“鳝鱼”后疼痛较前加重
10195259 未进食
20014750 未进饮食
20023741 未进食
0 547
1 887
2 3
3 31092
3000
0 1131
1 1935
2 4
3 7642
3000


In [7]:
from run import train1
best_mdl_path = train1(file_path=r'data\厌食.txt', text_field=0, label_field=1, log_file=r'output/logs/厌食_train_20220811.txt')
print(best_mdl_path)

DEBUG:root:Loading data lines...
DEBUG:root:6001 Lines Loaded!
DEBUG:root:Statistic Class Sample Number...
DEBUG:root:Class Sample Number: {'2': 2000, '0': 2000, '1': 2000}
INFO:root:Initializing Class TextClassifier...


2, total num: 2000, train num: 1600
0, total num: 2000, train num: 1600
1, total num: 2000, train num: 1600


Some weights of the model checkpoint at ../../BertModels/medical-roberta-wwm were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.decoder.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertModel were not initialized from the model checkpoint at ../../BertModels/medical-roberta-wwm and are newly initialized: ['bert.pooler.dense.bias

KeyboardInterrupt: 